# Matemática para Ciencia de los Datos
- Documento base:  https://nbviewer.jupyter.org/github/buruzaemon/svd/blob/master/01_SVD_visualizing_data.ipynb


# Singular Value Descomposition (SVD)

Singular value decomposition factoriza una matriz $X \in \mathbb{R}^{m \times n}$  en:

* Matriz $U \in \mathbb{R}^{m \times m}$ son los vectores singular-izquierdos de $X$, donde las columnas son el conjunto de auto-vectores ortonormales de  $X \, X^{T}$.
* Matriz diagonal $\Sigma$ con entradas $\sigma \in \mathbb{R}$ que son los valores singulares  no-negativos de $X$.
* Matriz $V \in \mathbb{R}^{n \times n}$ son los vectores singular-derechos de $X$, donde las columnas son el conjunto de auto-vectores ortonormales de  $X^{T} \, X$.

tal que, 

\begin{align}
  X &= U \, \Sigma \, V^{T}
\end{align}

Las dimensiones de cada una de las matrices que factorizan $X \in \mathbb{R}^{m \times n}$ son:

![](https://drive.google.com/uc?id=1sVuy8_b_P9hZJF9zsrpB3j6d0EithUtR)


Con respecto a la relación con los auto-valores y auto-vectores: 

* $U$ corresponde a los auto-vectores de $X \, X^{T}$
* $V$ corresponde a los auto-vectores de $X^{T} \, X$
* $\Sigma$ corresponde a los auto-valores de $X \, X^{T}$ o $X^{T} \, X$, que son los mismos.


## Ejemplo: Conjunto de datos Iris

[Iris flower dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set). 

Los datos son multivariados, con 150 mediciones de 4 atributos (length and width cm de ambos:  sepal and petal) en tres distintas especies de Iris. De las 150 mediciones, hay 50 mediciones para _Iris setosa_, _Iris versicolor_, e _Iris virginica_.

[Scikit Learn's `datasets`](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) incluye el conjunto de datos Iris.

![](https://drive.google.com/uc?id=125KRc9v-vQ5dVfO2IR7jkqyRChoDtCHI)

Fuente: https://www.researchgate.net/publication/265877256_How_plants_grow_and_move/figures?lo=1


In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.stats
import sklearn.datasets
import sklearn.preprocessing



In [ ]:
iris = sklearn.datasets.load_iris()

X = pd.DataFrame(iris.data, columns=iris.feature_names)

X_zscaled = (X - X.mean()) / X.std(ddof=1)

Y = pd.DataFrame(iris.target, columns=['target'])
Y['species'] = Y.apply(lambda r: iris.target_names[r])

print("Dimensiones de la matriz: (filas,columnas)", X.shape)
print(X.head(5))
print(X_zscaled )

In [ ]:
U_iris, S_iris, Vt_iris = np.linalg.svd(X_zscaled)

#### $U$: vectores singular-izquierdos de $X$

Las filas de $U$ corresponden a las filas de la matriz original de datos $X$, mientras que las columnas son el conjunto de auto-vectores ordenados, ortonormales de  $X \, X^{T}$.

In [ ]:
print('La matriz U tiene {} filas, {} columnas\n'.format(*U_iris.shape))

print('{}'.format(pd.DataFrame(U_iris).head(5)))

#### $V$: vectores sigular-derechos de $X$

[`numpy.linalg.svd`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html) retorna  $V^{T}$ en lugar de $V$, tal que las columnas de $V^{T}$ corresponden a la matriz orginal de datos de $X$. Y donde las filas son el conjunto de auto-vectores ordenados, ortonormales de $X^{T} \, X$.

In [ ]:
print('La matriz Vt tiene {} filas, {} columnas\n'.format(*Vt_iris.shape))

print('{}'.format(pd.DataFrame(Vt_iris).head()))

#### $\Sigma$: valores singulares de $X$

Los elementos $\sigma_{i}$ de la matriz diagonal $\Sigma$ son los valores sigulares  no-negativos de la matriz $X$, los cuales son realmente las raíces cuadradas de los auto-valores no-zero de $X^{T} \, X$ (y también para $X \, X^{T}$). Estos valores singulares pueden ser usados para determinar la cantidad de varianza $X^{\prime}$ explicada de la matriz de datos original $X$ cuando reducimos las dimensiones para encontrar una aproximación de rango inferior.

\begin{align}
   X^{\prime}_{k} &=  U_{k} \, \Sigma_{k} \, V^{T}_{k} \\
                           &\approx X_{r} & \text{ donde } rank(X^{\prime}) = k \lt rank(X) = r
\end{align}

La cantidad de varianza que la aproximación del rango reducido  $X^{\prime}_{k}$ explica de  $X_{r}$ es:

\begin{align}
  \text{suma de varianza explicada} &= \frac{\sum_{j=1}^{k} \sigma_{j}^{2}}{\sum_{i=1}^{r} \sigma_{i}^{2}}
\end{align}

**NOTE**: [`numpy.linalg.svd`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html) realmente retorna un $\Sigma$ que no es una matriz diagonal, sino una lista de las entradas de la diagonal.

In [ ]:
print("Vector Sigma tiene: ", len(S_iris), " elementos")
print(S_iris)

In [ ]:
num_sv_iris = np.arange(1, S_iris.size+1)

cum_var_explained_iris = [np.sum(np.square(S_iris[0:n])) / np.sum(np.square(S_iris)) for n in num_sv_iris]

Se grafica la suma de la varianza explicada como una función del número de valores singulares usados cuando reducimos el rango para encontrar una matriz de rango-inferior $X^{\prime}$ para aproximar $X$. Lo cual da pistas de cuántas dimensiones se podrían usar.

In [ ]:
fig = plt.figure(figsize=(7.0,5.5))
ax = fig.add_subplot(111)

plt.plot(num_sv_iris,
         cum_var_explained_iris,
         color='#2171b5',
         label='varianza explicada',
         alpha=0.65,
         zorder=1000)

plt.scatter(num_sv_iris,
            sklearn.preprocessing.normalize(S_iris.reshape((1,-1))),
            color='#fc4e2a',
            label='valores singulares (normalizados)',
            alpha=0.65,
            zorder=1000)

plt.legend(loc='center right', scatterpoints=1, fontsize=8)

ax.set_xticks(num_sv_iris)
ax.set_xlim(0.8, 4.1)
ax.set_ylim(0.0, 1.1)
ax.set_xlabel(r'Número de valores singulares utilizados')
ax.set_ylabel('Varianza en los datos explicada')
ax.set_title('Conjunto de datos Iris: suma de la varianza explicada & valores singulares',
             fontsize=14,
             y=1.03)

ax.set_facecolor('0.98')

plt.grid(alpha=0.8, zorder=1)
plt.tight_layout()

#### Reducción de dimensión

A juzgar por la curva que representa la suma de la varianza explicada en la figura anterior, se puede ver que:

* con 2 valores singulares, cerca de 96.5% de la varianza de $X$ puede ser explicada
* con 3 valores singulares, ese número llega hasta aproximadamente 99.8%

Ahora se grafica utilizando los primeros 2 valores singulares para representar los datos en los ejes $x$ y $y$, respectivamente.

In [ ]:
idx_setosa = np.where(iris.target==0)[0]
idx_versicolor = np.where(iris.target==1)[0]
idx_virginica = np.where(iris.target==2)[0]

setosa_x = U_iris[idx_setosa, 0]
setosa_y = U_iris[idx_setosa, 1]

versicolor_x = U_iris[idx_versicolor, 0]
versicolor_y = U_iris[idx_versicolor, 1]

virginica_x = U_iris[idx_virginica, 0]
virginica_y = U_iris[idx_virginica, 1]

In [ ]:
fig = plt.figure(figsize=(7.0,5.5))
ax = fig.add_subplot(111)

plt.scatter(setosa_x,
            setosa_y,
            marker='o',
            color='#66c2a5',
            label='Iris-setosa',
            zorder=1000)

plt.scatter(versicolor_x,
            versicolor_y,
            marker='D',
            color='#fc8d62',
            label='Iris-versicolor',
            zorder=1000)

plt.scatter(virginica_x,
            virginica_y,
            marker='^',
            color='#8da0cb',
            label='Iris-virginica',
            zorder=1000)

plt.legend(loc='upper left', scatterpoints=1, fontsize=8)

ax.set_xlabel(r'valor singular $\sigma_{1}$')
ax.set_ylabel(r'valor singular $\sigma_{2}$')
ax.set_title('Gráfico 2D del conjunto de datos Iris',
             fontsize=14,
             y=1.03)
ax.set_facecolor('0.98')

plt.grid(alpha=0.6, zorder=1)
plt.tight_layout()

**Observación final:**

Se logra visualizar el conjunto de datos original que estaba en 4D en 2D usando las primeras dos columnas de valores singulares de $U$, matriz de vectores singular-izquierda, se logra ver que hay una clara separación para las clases _Iris setosa_ y las otras. De otra manera, la diferenciación entre _Iris versicolor_ e _Iris virginica_ no parece clara.


## Ejemplo adicional en imágenes

In [ ]:
from PIL import Image

plt.style.use('ggplot')
img = Image.open('perritaejemplo.jpg')
print("Detalles de img: ", img)
# convert image to grayscale
imggray = img.convert('LA')
print("Detalles de imggray: ", imggray)
print(imggray)

# convert to numpy array
imgmat = np.array(list(imggray.getdata(band=0)), float)

# Reshape according to orginal image dimensions
imgmat.shape = (imggray.size[1], imggray.size[0])

print("\n\n")
print("Resolución de la imagen original: ", imgmat.shape)
print("\n\n")

plt.figure(figsize=(9, 6))
plt.imshow(imgmat, cmap='gray')
plt.show()

U, D, V = np.linalg.svd(imgmat)

print("\n\n")
print("Dimensiones de U:", U.shape)
print("Dimensiones de D:", D.shape)
print("Dimensiones de V:", V.shape)
print("\n\n")

for i in [5, 50, 200, 500, 1500, 2000]:
    #reconstimg = np.matrix(U[:, :i]) * np.diag(D[:i]) * np.matrix(V[:i, :])
    reconstimg = np.matmul( np.matmul( np.matrix(U[:, :i]) , np.diag(D[:i]) ) , np.matrix(V[:i, :]) )
    
    plt.imshow(reconstimg, cmap='gray')
    title = "n = %s" % i
    plt.title(title)
    plt.show()